Implement PKCS#7 padding
A block cipher transforms a fixed-sized block (usually 8 or 16 bytes) of plaintext into ciphertext. But we almost never want to transform a single block; we encrypt irregularly-sized messages.

One way we account for irregularly-sized messages is by padding, creating a plaintext that is an even multiple of the blocksize. The most popular padding scheme is called PKCS#7.

So: pad any block to a specific block length, by appending the number of bytes of padding to the end of the block. For instance,

"YELLOW SUBMARINE"
... padded to 20 bytes would be:

"YELLOW SUBMARINE\x04\x04\x04\x04"

In [15]:
def PKCS7(text, blockLength):
    padSize = blockLength - (len(text) % blockLength)
    return text + b'\x04' * padSize

plaintext = b'YELLOW SUBMARINE'
padded = PKCS7(plaintext, 20)
print(padded)

b'YELLOW SUBMARINE\x04\x04\x04\x04'


Implement CBC mode
CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

In [3]:
import base64
from Crypto.Cipher import AES
def AESdecrypt(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(ciphertext)

def AESencrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)

def XOR(byteString1, byteString2):
    result = b''
    for b1,b2 in zip(byteString1, byteString2):
        result += bytes([b1 ^ b2])
    return result

def CBCdecrypt(key, ciphertext, IV):
    blockSize = 16
    plaintext = b''
    cipherBlocks = [ciphertext[i:i + blockSize] for i in range(0,len(ciphertext), blockSize)]
    for i in range(len(cipherBlocks) - 1, -1, -1):
        decryptedBlock = AESdecrypt(key, cipherBlocks[i])
        previousBlock = IV if i == 0 else cipherBlocks[i-1]
        plaintext = XOR(previousBlock, decryptedBlock) + plaintext
    return plaintext

def CBCencrypt(key, plaintext, IV):
    blockSize = 16
    ciphertext = b''
    plaintextBlocks = [plaintext[i:i + blockSize] for i in range(0,len(plaintext), blockSize)]
    prevCipherBlock = IV
    for i in range(len(plaintextBlocks)):
        cipherBlock = AESencrypt(key, XOR(prevCipherBlock, plaintextBlocks[i]))
        prevCipherBlock = cipherBlock
        ciphertext += cipherBlock
    return ciphertext

key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
blockSize = 16

with open('set2_chall10.txt','r') as file:
    base64d = file.read().replace('\n','')
    ciphertext = base64.b64decode(base64d)
    plaintext = CBCdecrypt(key, ciphertext, IV)
    print(plaintext)
plaintext = b'hello world' + b'\x04' * 5
ciphertext = CBCencrypt(key, plaintext, IV)
print(ciphertext)
plaintext = CBCdecrypt(key, ciphertext, IV)
print(plaintext)



b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

An ECB/CBC detection oracle
Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:

encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.

In [35]:
import random
from Crypto.Cipher import AES

def PKCS7(text, blockLength):
    padSize = blockLength - (len(text) % blockLength)
    return text + b'\x04' * padSize

def AESencrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)

def randomAESKey(size):
    key = b''
    for i in range(size):
        randNum = random.randint(0, 255)
        randByte = bytes([randNum])
        key += randByte
    return key

key = randomAESKey(16)
print(key, len(key))

def AESencryptWithRandomKey(data):
    return AESencrypt(randomAESKey(16))

def encryption_oracle(data):
    frontPaddingSize = random.randint(5, 10)
    frontPadding = randomAESKey(frontPaddingSize)
    backPaddingSize = random.randint(5,10)
    backPadding = randomAESKey(backPaddingSize)
    data = frontPadding + data + backPadding
    data = PKCS7(data, 16)
    blockCipherMode = random.randint(0,1)
    key = randomAESKey(16)
    IV = randomAESKey(16)
    ciphertext = b''
    if blockCipherMode == 0:
        ciphertext = CBCencrypt(key, data, IV)
    else:
        ciphertext = AESencrypt(key, data)
    return ciphertext, blockCipherMode

def countRepeatedBlocks(ciphertext, blockSize):
    blocks = [ciphertext[i:i+blockSize] for i in range(0, len(ciphertext), blockSize)]
    matchingBlockCount = len(blocks) - len(set(blocks))
    return matchingBlockCount

plaintext = b'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
numECB = 0
numCBC = 0
success = 0
for i in range(50):
    ciphertext, mode = encryption_oracle(plaintext)
    numRepeated = countRepeatedBlocks(ciphertext, 16)
    if numRepeated > 0:
        numECB += 1
    else:
        numCBC += 1
    if mode == int(numRepeated > 0):
        success += 1
print('# ECB ciphertexts: {0}'.format(numECB))
print('# CBC ciphertexts: {0}'.format(numCBC))
print('# successful cipher mode identifications: {0}'.format(success))

b'.\xdf\x073[\xfe\xdb\x9e\\n\xab\xec/v\x81\x9b' 16
# ECB ciphertexts: 32
# CBC ciphertexts: 18
# successful cipher mode identifications: 50


Byte-at-a-time ECB decryption (Simple)
Copy your oracle function to a new function that encrypts buffers under ECB mode using a consistent but unknown key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string:

Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
Spoiler alert.
Do not decode this string now. Don't do it.

Base64 decode the string before appending it. Do not base64 decode the string by hand; make your code do it. The point is that you don't know its contents.

What you have now is a function that produces:

AES-128-ECB(your-string || unknown-string, random-key)
It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

Here's roughly how:

Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
Detect that the function is using ECB. You already know, but do this step anyways.
Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
Repeat for the next byte.
Congratulations.
This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can decrypt the contents of those ciphertexts, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.

In [94]:
import random
import base64 

def PKCS7(text, blockLength):
    padSize = blockLength - (len(text) % blockLength)
    return text + b'\x04' * padSize

def AESencrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)

def randomAESKey(size):
    key = b''
    for i in range(size):
        randNum = random.randint(0, 255)
        randByte = bytes([randNum])
        key += randByte
    return key

key = randomAESKey(16)
print(key, len(key))

def AESencryptWithRandomKey(data):
    return AESencrypt(randomAESKey(16))

def encryption_oracle(data, key):
    suffix_b64 = 'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'
    suffix = base64.b64decode(suffix_b64)
    data = data + suffix
    data = PKCS7(data, 16)
    return AESencrypt(key, data)
    
def countRepeatedBlocks(ciphertext, blockSize):
    blocks = [ciphertext[i:i+blockSize] for i in range(0, len(ciphertext), blockSize)]
    matchingBlockCount = len(blocks) - len(set(blocks))
    return matchingBlockCount

guess = b'A'
for i in range(1,20):
    guess = b'A' * i
    ciphertext = encryption_oracle(guess, key)
    print(i, len(ciphertext))

guess = b'A' * 50
count = countRepeatedBlocks(encryption_oracle(guess, key), 16)
if count > 0:
    print('ECB')
else:
    print('CBC')
    


blockSize = 16

targetLength = len(encryption_oracle(b'', key))
plaintext = b''
offsetString = bytearray((blockSize-1) * b'A')
guessString = bytearray(my_string)

for k in range(int(targetLength / 16)):
    for j in range(16):
        targetCiphertext = encryption_oracle(bytes(offsetString[:len(offsetString) - j]), key)
        for i in range(256):
            guess = bytes(guessString) + bytes([i])
            ciphertext = encryption_oracle(guess, key)
            if ciphertext[:blockSize] == targetCiphertext[blockSize*k:blockSize*(k+1)]:
                guessString += bytes([i])
                plaintext += bytes([i])
                del guessString[0]
                break
    guessString = bytearray(plaintext[len(plaintext) - blockSize +1:])
    #update guessString with known plaintext to start guessing next block
print(plaintext)

b';\xbd<\xa2\x05wr\x12\r\xb7T\xed\x8f\x92\xea-' 16
1 144
2 144
3 144
4 144
5 144
6 160
7 160
8 160
9 160
10 160
11 160
12 160
13 160
14 160
15 160
16 160
17 160
18 160
19 160
ECB
b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x04\x04\x04\x04\x04\x04"


ECB cut-and-paste
Write a k=v parsing routine, as if for a structured cookie. The routine should take:

foo=bar&baz=qux&zap=zazzle
... and produce:

{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
(you know, the object; I don't care if you convert it to JSON).

Now write a function that encodes a user profile in that format, given an email address. You should have something like:

profile_for("foo@bar.com")
... and it should produce:

{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
... encoded as:

email=foo@bar.com&uid=10&role=user
Your "profile_for" function should not allow encoding metacharacters (& and =). Eat them, quote them, whatever you want to do, but don't let people set their email address to "foo@bar.com&role=admin".

Now, two more easy functions. Generate a random AES key, then:

Encrypt the encoded user profile under the key; "provide" that to the "attacker".
Decrypt the encoded user profile and parse it.
Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile.

In [116]:
import random
from Crypto.Cipher import AES

def parseURLParameters(parameters):
    params = {}
    pairs = parameters.strip().split('&')
    for pair in pairs:
        name, value = pair.split('=')
        params[name] = value
    return params

def encodeParams(params):
    url = ''
    for param in params:
        url += param + '=' + params[param] + '&'
    return url[:-1]

url = 'foo=bar&baz=qux&zap=zazzle'
params = parseURLParameters(url)
print(params)

def profile_for(email):
    profile = {}
    profile['email'] = email
    profile['uid'] = '10'
    profile['role'] = 'user'
    return encodeParams(profile)

def PKCS7(text, blockLength):
    padSize = blockLength - (len(text) % blockLength)
    return text + b'\x04' * padSize

def AESencrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)

def AESdecrypt(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(ciphertext)

def randomAESKey(size):
    key = b''
    for i in range(size):
        randNum = random.randint(0, 255)
        randByte = bytes([randNum])
        key += randByte
    return key

blockSize = 16
key = randomAESKey(blockSize)
def encryptedUserProfile(email):
    profile = bytes(profile_for(email), 'utf-8')
    return AESencrypt(key, PKCS7(profile, blockSize))

def decryptUserProfile(data):
    params = AESdecrypt(key, data)
    # remove padding
    params = bytearray(params)
    index = params.find(b'\x04')
    if index != -1:
        params = params[index:]
    params = params.decode('utf-8')
    userProfile = parseURLParameters(params)
    return userProfile

profile = profile_for('robzom@gmail.com')
print(profile)

for i in range(16):
    email = 'A' * i
    ciphertext = encryptedUserProfile(email)
    print(ciphertext, len(ciphertext), str(i))
email = 'A'*10 + 'admin' + ' '*11 + ' '*3 #&uid=10&role=user
ciphertext = encryptedUserProfile(email)
#admin in second block
adminBlock = ciphertext[blockSize: blockSize*2]
email = 'mea@gmail.com'
ciphertextFinal = encryptedUserProfile(email)
ciphertextFinal = bytearray(ciphertextFinal)
ciphertextFinal[-16:] = adminBlock
ciphertextFinal = bytes(ciphertextFinal)
profile = decryptUserProfile(ciphertextFinal)
print(profile)

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}
email=robzom@gmail.com&uid=10&role=user
b'\x83\xcd\xbe\xd9\x81&U.\x93P\x87\xfa\xee\x06QS\\w0\x10\x18\x8d\xa0\xff\x03\xd7\xbd\xde\x18I\xf2\xd2' 32 0
b'QnDuy\xb8}\xaa\xfe\x84\xfci`\x11\x98\xd2\xe1\xab\x07\xd6ONP\x80\xe8\x8f\x14()\xeb\xe6u' 32 1
b'\xeaQ\x14\xe0B#\xf5\x8d\x01\x00\xc4\xbf\xce\x9f\x11\x8f;&"\x8d\x89F\xb6\xf6\xf8\x1f\x85&B\x91\x840' 32 2
b'p\x8e\xbf[)\x80i)V\x94\xec\xc67]\xb9\xcb\x8c\xc8\x0e+\x0eTv~v\xb0|\xa5\xcc5\re' 32 3
b'\xea@g\x1b\x1c\xb4W]v\x1fBV\x8b\xa7\x82?\x94\xc6\x13%9\x8f\xbc\x8eC\xab\xdf\xa0\xe6\xff}o' 32 4
b'\xe3\x19\x9f\x0b\x00\x9d\xf0\xb5\xc3\xef\xa1?\xeds>\x06\xc08\xb3 \x0c\x99\x9e}7\x90=\xc2\xf7\xf9h\xc7' 32 5
b'4\x91\x95Y\xf7\x96\xa4\xe5y\xe8\xfc4\xd9\xe1\xd1\x925\xd2\xac\xb3\xf1\xd0\xf6@F<e\xf6\xd11Hu' 32 6
b'Cu\xef\xac\xce2/\xbf\x95{tf^R1P)\xde\xa6b\xa9\x10\xe8\xdfv2\x04\xb9h\x95\xbe>' 32 7
b'M\xd1\xc9-\x0e\r0;L\x04M \xea\x0e\xb7\xb9\xe9\xc2\x9c\xf9[\xa3nzh-&\xd2\x14\xa2/\xad' 32 8
b'\xabz\xdd\x98(\xc68a\xec\xb

Byte-at-a-time ECB decryption (Harder)
Take your oracle function from #12. Now generate a random count of random bytes and prepend this string to every plaintext. You are now doing:

AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)
Same goal: decrypt the target-bytes.

Stop and think for a second.
What's harder than challenge #12 about doing this? How would you overcome that obstacle? The hint is: you're using all the tools you already have; no crazy math is required.

Think "STIMULUS" and "RESPONSE".